In [ ]:
import random

class TicTacToe:
  def __init__(self , player1 , player2):
    self.player1 = player1
    self.player2 = player2
    self.board = [' '] * 9
    self.player1_turn = random.choice([True , False])

  def check_full(self , ):
    emptys = [x for x in self.board if x == ' ']
    if len(emptys) == 0:
      return True
    return False

  def print_board(self , ):
    b  = self.board
    board = f"""
        {b[0]} | {b[1]} | {b[2]}
        --|---|---
        {b[3]} | {b[4]} | {b[5]}
        --|---|---
        {b[6]} | {b[7]} | {b[8]}
    """

    print (board)



  def check_winner(self , char):
    b = self.board
    winner = False
    conditions = (
        (0 , 1 , 2) , (3 , 4 , 5) , (6 , 7 , 8) ,
        (0 , 3 , 6) , (1 , 4 , 7) , (2 , 5 , 8) ,
        (0 , 4 , 8) , (2 , 4 , 6)
    )


    for each_conditions in conditions:
      i1 , i2 , i3 = each_conditions
      if char == b[i1] == b[i2] == b[i3]:
        winner = True
        return winner

    return winner

  def play(self , ):
    self.player1.new_game()
    self.player2.new_game()


    while True:
      if self.player1_turn:
        player = self.player1
        other_player = self.player2
      else:
        player = self.player2
        other_player = self.player1
      self.print_board()
      #Ask for move
      if player.player_type == 'human':
        print(f'{player.name} Turn')
        self.print_board()


      move = player.make_move(self.board)


      #check validity
      if (move < 0 or move > 8) or (self.board[move] != ' '):
        player.reward(-0.25 , self.board)
        print('Invalid Move!!!')
        continue
      #put char
      self.board[move] = player.char
      #check winner
      win =self.check_winner(player.char)
      if win :
        player.winner()
        player.reward(5 , self.board)
        other_player.reward(-5 , self.board)
        self.print_board()
        break
      #check full(draw)
      full = self.check_full()
      if full:
        print('Game ended as draw')
        player.reward(0.5 , self.board)
        other_player.reward(0.5 , self.board)
        self.print_board()
        break
      #swap turn
      player.reward(0.25 , self.board)
      self.player1_turn = not self.player1_turn

In [ ]:
class Player:
  def __init__(self , name , char , player_type = 'human'):
    self.name = name
    self.char = char
    self.player_type = player_type


  def make_move(self , board):
    move = input('Make your move: ')
    return int(move)


  def new_game(self,):
    print(f'{self.name} is {self.char}')


  def winner(self , ):
    print(f'{self.name} is winner')


  def reward(self , reward_value , board):
    print(f'{self.name} gets {reward_value}')

In [ ]:
class AI(Player):
  def __init__(self , name , char , epsilon = 1 , gamma = 0.9 , alpha = 0.01):
    super().__init__(name , char , player_type = 'AI')
    self.epsilon = epsilon # Espilon Greddy
    self.gamma = gamma # Discount Factor
    self.alpha = alpha # Learing Rate
    self.q_table = {}

  def get_Q(self , state , action):
    if self.q_table.get((state , action)) == None:
      self.q_table[(state , action)] = 5
    return self.q_table[(self , action)]

  def aviable_moves(self , board):
    return [x for x in range(9) if board[x] == ' ' ]

  def q_learn(self , state , action , reward , new_state):
    '''
    new_q_value = prev_q_value + alpha * (reward + gamma * max_q - prev_q_value)
    '''
    prev_q_value = self.get_Q(state , action) # make this reward
    aviable_actions = self.aviable_moves(state) # make this reward
    aviable_q_values = [self.get_Q(new_state , action ) for action in aviable_actions]
    max_q = max(aviable_q_values)
    self.q_table[(state , action)] = prev_q_value + self.alpha * (reward + self.gamma * max_q - prev_q_value)

  def reward(self, reward_value , board):
    self.q_learn(self.prev_board ,
                 self.prev_action ,
                 reward_value ,
                 tuple(board))

  def make_move(self , board):
    # Save old state
    self.prev_board = tuple(board)
    aviable_action = self.aviable_moves(board)
    #Select Random Action(Exploration)
    if random.random() < self.epsilon:
      action_taken = random.choice(aviable_action)
      self.prev_action = action_taken
      self.epsilon *= 0.9999999999 # Epsilon decay
      return action_taken

    #Selecting action with max Q-Vlalue(Exploitation)
    q_values = []
    for actions in aviable_action:
      q_values.append(self.get_Q(board , actions))
    max_q_value = max(q_values)
    index = q_values.index(max_q_value)
    action_taken = aviable_action[index]
    return action_taken

In [ ]:
p1 = AI('P-One' , 'X')
p2 = AI('p-Two' , 'O')

In [ ]:
for i in range(5):
  print(f'\nEpisode {i + 1}')
  game = TicTacToe(p1 , p2)
  game.play()


Episode 1
P-One is X
p-Two is O

          |   |  
        --|---|---
          |   |  
        --|---|---
          |   |  
    


KeyError: ignored